In [2]:
# リスト 5.4.1 事前学習済みfastText利用サンプル（説明コメント付き）

# ▼ 目的
#  - 事前学習済み fastText（もしくは word2vec 互換フォーマット）のテキストベクトルファイル（.vec）
#    を Gensim の KeyedVectors として読み込み、個別語のベクトル参照・類似語検索・アナロジー演算
#   （“日本:東京 = フランス:X”）を行う最小実行例である。
#  - fastText の .vec は「word2vec テキスト形式（ヘッダ + 語彙行）」で、各行が
#      <token> <f1> <f2> ... <fD>
#    のような空白区切りの浮動小数列になっている。binary=False を指定して読み込む。

# ▼ 注意点（理論・実装）
#  - KeyedVectors は「語→ベクトル」の読み取り専用コンテナで、学習（訓練）は含まない。
#  - 類似度は既定で「コサイン類似度」。most_similar は
#      sim(w, a - b + c) を最大化する語を返す（a,b,c は positive/negative の線形合成）。
#    ここで sim(u,v) = (u・v) / (||u|| ||v||)。
#  - OOV（語彙外）を参照すると KeyError。必要なら try/except で防御する（本サンプルは簡潔化のため未実装）。
#  - 学習済み fastText（サブワードあり）の場合でも .vec にエクスポートされた段階では
#    事前計算済み語彙ベクトルを読み込む（未知語サブワード合成は fastText 本体の推論機能で実現される）。

# ▼ 実務メモ
#  - 「表記ゆれ」（全角/半角，旧字体，新仮名遣い）により OOV になりがち。正規化（NFKC 等）を推奨。
#  - 日本語 fastText 事前学習モデルは Wikipedia ベースが多く、口語/専門語では分布ずれが起きることがある。


# モデルファイルのロード
import gensim

# 'model.vec' は word2vec 互換の「テキスト形式」想定（binary=False）。
# ファイル先頭行は「語彙数 次元数」のヘッダ、以降は "語 次元値..." の行が続く。
# 例） fastText の 'cc.ja.300.vec' などを指定可能（ただし大容量に注意）。
model = gensim.models.KeyedVectors.load_word2vec_format("model.vec", binary=False)
print(model)  # 語彙数やベクトル次元などの概要が表示される

# 「世間」の特徴量ベクトルを調べる
# - ベクトルは実数配列（R^d）。連続空間上の位置として語の意味分布を表現する（分布仮説）。
# - モデルに語が存在しない場合は KeyError になるため、実務では try/except での防御が望ましい。
print("「世間」の特徴量ベクトル")
print(model["世間"])

# 「世間」の類似語を調べる
# - most_similar("世間") は、コサイン類似度が高い順に近傍語を返す。
# - 返値は (語, 類似度) のタプル。類似度は [-1,1] の範囲で、1 に近いほど方向が一致。
print()
print("「世間」の類似語")
for item, value in model.most_similar("世間"):
    print(item, value)

# 「日本」->「東京」から「フランス」->「X」を求める
# - アナロジー： argmax_w sim(w, vec("東京") - vec("日本") + vec("フランス"))
# - 「国→首都」の関係ベクトル（translation）を転用して、フランスの首都（パリ）に近い語を推定する。
# - 分布ベクトルの線形性（王 - 男 + 女 ≈ 女王 等）を利用する古典的検証で、完全ではない点に注意。
model.most_similar(positive=["東京", "フランス"], negative=["日本"])

KeyedVectors<vector_size=300, 351122 keys>
「世間」の特徴量ベクトル
[-1.1506e-01 -1.0583e-01 -2.4785e-01  4.7711e-02 -2.6220e-02  1.4092e-01
  1.9899e-01  1.4590e-01 -1.0638e-01  9.8768e-02  1.4652e-02 -5.8634e-02
  2.6814e-01 -1.0366e-01  3.5728e-01 -2.9486e-02 -7.3661e-02 -2.8939e-01
  2.7001e-01 -2.3144e-01 -5.9499e-02  2.2983e-01  1.9749e-01  2.0851e-02
  2.9852e-02  8.5426e-02  1.2522e-01  6.8352e-02  1.8051e-01 -5.1769e-02
  1.1419e-01 -1.6506e-01 -3.1969e-01  5.0779e-01 -5.8382e-02  2.5845e-01
  1.5132e-01  1.8439e-01  8.1836e-03  5.0382e-02 -4.9251e-02  5.4424e-02
 -6.1879e-03  1.7422e-01  2.6872e-01 -2.3755e-01 -8.8048e-02 -1.1178e-01
 -1.3528e-01 -4.5117e-01 -1.4435e-01 -3.0919e-01 -1.6905e-01  1.5945e-01
  9.6494e-02  3.0769e-01 -7.4793e-02 -1.6989e-01  2.4768e-01 -2.5647e-02
  3.7486e-01 -2.5074e-02  1.7024e-02  2.2682e-01 -2.7441e-01  3.8925e-01
 -1.3320e-03 -1.8967e-01  6.9611e-02  2.4771e-01  7.4916e-02 -1.5602e-03
 -1.0169e-01  2.5634e-01 -2.0373e-01 -6.3139e-02 -1.7744e-01 -1.0650

[('パリ', 0.6681745648384094),
 ('トゥールーズ', 0.5696516036987305),
 ('コートダジュール', 0.5662188529968262),
 ('パリ郊外', 0.5582724809646606),
 ('ストラスブール', 0.557762086391449),
 ('リヨン', 0.5561243295669556),
 ('サン＝クルー', 0.551009476184845),
 ('ディジョン', 0.5456872582435608),
 ('ボルドー', 0.5447909235954285),
 ('マルセイユ', 0.5429384112358093)]